In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import numpy as np
import pandas as pd
from decimal import Decimal
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#设置print格式
np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(threshold=20)
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
pd.set_option('display.max_rows', 500)  #最大行数
pd.set_option('display.max_columns', 500)    #最大列数
pd.set_option('display.width', 4000)        #页面宽度

def convert_num(x, fill=None):
    x = str(x)
    try:
        res = Decimal(x)/ Decimal(10000)
    except Exception:
        res = fill if fill else np.NaN
    return res

#数据路径
import os
url_base = os.getcwd()

E:\ANACONDA\envs\python374\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\ANACONDA\envs\python374\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\ANACONDA\envs\python374\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\ANACONDA\envs\python374\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

E:\ANACONDA\envs\python374\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\ANACONDA\envs\python374\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\ANACONDA\envs\python374\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\ANACONDA\envs\python374\lib\site-packages\tensorboard\compat\tensorflow_stub

In [2]:
#引入数据
# /`CSE-CIC-IDS2018` all Data  
url_data = f"{url_base}/all.csv"

# info data (column names, col types)
url_info = f"{url_base}/all_names.names"
  
# Import info data
df_info = pd.read_csv(url_info, sep=":", skiprows=1, index_col=False, names=["colname", "type"])

colnames = df_info.colname.values
coltypes = np.where(df_info["type"].str.contains("continuous"), "float", "str")
# print(colnames)

# Import data
# 读取并且指定数据类型dtype,names指定列名
df = pd.read_csv(url_data, names=colnames, index_col=False, dtype=dict(zip(colnames, coltypes)))

# print(df)

# Dumminize
X = pd.get_dummies(df.iloc[:,:-1]).values
print(np.shape(X))
# print(X)
# X1 = pd.get_dummies(df.iloc[:,:-1])
# print(X1)

for x1 in X:
    if np.isinf(x1).any():
        print(x1)
        for i in range(len(str(x1))):
            if np.isinf(x1[i]):
                print(i)
        # print(np.isinf(x1).any())

# Create Traget Flag
# Anomaly data when status is normal, Otherwise, Not anomaly.
y = np.where(df.Label == "BENIGN", 1, 0)
# print(y)

# Split Data
# 训练数据和测试数据对半开
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)
print(np.shape(X_train))
print(y_test)

(1048568, 78)
(524284, 78)
[1 1 1 ... 1 1 0]


In [3]:
#数据预处理

#标准化
scaler = StandardScaler()
X_soc = X_train
X_soc_test = X_test
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 标准化模型保存
from sklearn.externals import joblib
joblib.dump(scaler, 'CSE-CIC-IDS2018_StandardScaler_01')

X_train = np.reshape(X_train,[-1,78,1])
X_test = np.reshape(X_test,[-1,78,1])

# one_hot编码
def dense_to_one_hot(labels_dense, num_classes):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset+labels_dense.ravel()] = 1
    return labels_one_hot
 
num_classes  = 2
one_hot_train = dense_to_one_hot(y_train,num_classes)
one_hot_test = dense_to_one_hot(y_test,num_classes)
print(one_hot_train.shape)
print(one_hot_test)

(524284, 2)
[[ 0.000  1.000]
 [ 0.000  1.000]
 [ 0.000  1.000]
 ...
 [ 0.000  1.000]
 [ 0.000  1.000]
 [ 1.000  0.000]]


E:\ANACONDA\envs\python374\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [4]:
#建立模型结构，就是几个全连接
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, max_pool_1d,global_max_pool
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

# Building convolutional network
network = input_data(shape=[None,78,1], name='input')
# network = fully_connected(network, 64, activation='tanh')
network = fully_connected(network, 32, activation='tanh')
network = fully_connected(network, 10, activation='tanh')
network = dropout(network, 0.5)

network = fully_connected(network, 2, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
# Training
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit({'input': X_train}, {'target': one_hot_train}, n_epoch=2,
          validation_set=({'input': X_test}, {'target': one_hot_test}),
          show_metric=True, run_id='dnn_demo')

Training Step: 16383  | total loss: 0.06220 | time: 30.533s
| Adam | epoch: 002 | loss: 0.06220 - acc: 0.9882 -- iter: 524224/524284
Training Step: 16384  | total loss: 0.05832 | time: 37.354s
| Adam | epoch: 002 | loss: 0.05832 - acc: 0.9894 | val_loss: 0.02104 - val_acc: 0.9950 -- iter: 524284/524284
--


In [ ]:
#提取tflearn生成会话中的变量

input_ = model.session.graph.get_tensor_by_name('input/X:0')
output_ = model.session.graph.get_tensor_by_name('FullyConnected_2/BiasAdd:0')
softmax_ = model.session.graph.get_tensor_by_name('FullyConnected_2/Softmax:0')
target_ = model.session.graph.get_tensor_by_name('target/Y:0')
#构造损失函数
loss_ = tf.nn.softmax_cross_entropy_with_logits(logits=softmax_, labels=target_)#output_
#loss梯度
gradw, = tf.gradients(loss_,input_)

print(gradw)
#向前梯度
derivative, = tf.gradients(output_[0,1], input_)

In [ ]:
#将生成的对抗样本合法化
def x1_transform(x1):
    x1 = scaler.inverse_transform(x1)
    x1x = []
    x1x1 = np.float32(x1[0])
    x1x  =  np.append(x1x,x1x1)
    x1x = np.around(x1x, decimals=2)
    x1 = scaler.transform(np.reshape(x1x,[1,78]))
    return x1,x1x

    #x1是归一化的，x1x才是生成对抗样本

In [ ]:
def Deepfool(sample_n): 
    targets = [0,1]
    sample = np.array(X_train[sample_n:sample_n+1])
    x1 = np.reshape(sample,[1,78])
    # print(x1)
    
    #最大最小值
    x1_d = np.min(X_soc,axis=0)
    x1_u = np.max(X_soc,axis=0)
    
    #初始参数
    r_tot = np.zeros([1,78])
    # c = np.zeros(x1.shape[1]-34)

    for epoch in range(100):
        #计算梯度
        x_ = np.reshape(x1,[-1,78,1])
        y1,_,gg_ = model.session.run([softmax_,loss_,gradw],{input_:x_,target_:[targets]})

        if np.argmax(y1) == np.argmax(targets):
            #如果生成了对抗样本。
            
            x1,x1x = x1_transform(x1)
            #将生成的对抗样本合法化
            
            x_ = np.reshape(x1,[-1,78,1])
            y1,_,gg_ = model.session.run([softmax_,loss_,gradw],{input_:x_,target_:[targets]})
                      
            if np.argmax(y1) == np.argmax(targets):
                #如果生成的对抗样本合法化后依然是对抗样本。
                
                d_d = x1x-X_soc_test[sample_n]
                # print(d_d)
                L0 = np.linalg.norm(d_d,ord=0)
                print('\nL0=',L0)
                break #生成成功退出循环
                
                
        #用deepflow的方式计算更改样本
        gg=[]
        
        #gg = np.append(gg_[0][0:78],c)
        gg = gg_[0][0:78]
        gg = np.reshape(gg,[1,78])
        pert = np.abs(y1[0][np.argmax(targets)])/np.linalg.norm(gg)
        r_i = (pert+1e-8) * gg/np.linalg.norm(gg)
        r_tot = np.float32(r_tot+r_i)
        x1 = x1-1.1*r_tot
        
        
        #对x1的特征进行一个限制，限制在所有样本最大值内。
        x1 = scaler.inverse_transform(x1)
        x1x = []
        for i in range(78):
            x1x1 = np.float32(np.clip(x1[0][i:i+1],x1_d[i],x1_u[i]))
            x1x  =  np.append(x1x,x1x1)
        x1  = np.reshape(x1x,[1,78])
        x1 = scaler.transform(np.reshape(x1x,[1,78]))
        
    if epoch == 99:
        print('对抗样本生成失败，迭代次数 =',epoch, '\n')
        return False
    else:
        print('对抗样本生成成功，迭代次数 =',epoch,' , L0 =',L0, '\n')
        return x1x

In [ ]:
#实现saliency_map
def saliency_map(derivative, mask):
        mask = np.reshape(mask,[78])
        #print('mask',mask[0],mask[0,21,0])
        alphas = derivative * mask
        
        # pixel influence on sum of residual classes
        betas = -np.ones_like(alphas)
        
        sal_map = np.abs(alphas) * np.abs(betas) * np.sign(alphas * betas)
        # find optimal pixel & direction of perturbation
        #print(sal_map*1000)
        ''''''
        for sa in range(len(sal_map)):
            if mask[sa] ==0:
                sal_map[sa] = float("inf")#10000#
        # print(sal_map.shape)
                    
        idx = np.argmin(sal_map)
        #print('sal_map=', sal_map)
        #转换成(p1,p2)格式      
        idx = np.unravel_index(idx, sal_map.shape)
        pix_sign = np.sign(alphas)[idx]
        
        return idx[1], pix_sign

In [ ]:
def JSMA(sample_n):
    targets = [0,1]
    sample = np.array(X_train[sample_n:sample_n+1])
    x1 = np.reshape(sample,[1,78])
    #print(x1)
    
    mask = np.ones_like(x1)
    #最大最小值
    x1_d = np.min(X_soc,axis=0)
    x1_u = np.max(X_soc,axis=0)
    #定义边界
    max_=scaler.transform([x1_u])
    min_=scaler.transform([x1_d])
    max_ = np.reshape(max_,[78])
    min_ = np.reshape(min_,[78])

    # c = np.zeros(x1.shape[1]-34)
    L0 = 0

    for epoch in range(100):

        x_ = np.reshape(x1,[-1,78,1])
        y1,loss1,gg_,d = model.session.run([softmax_,loss_,gradw,derivative],{input_:x_,target_:[targets]})
        
        if np.argmax(y1) == np.argmax(targets):
            x1,x1x = x1_transform(x1)
            #将生成的对抗样本合法化

            x1 = scaler.transform(np.reshape(x1x,[1,78]))
            x_ = np.reshape(x1,[-1,78,1])
            y1,loss1,gg_ = model.session.run([softmax_,loss_,gradw],{input_:x_,target_:[targets]})

            if np.argmax(y1) == np.argmax(targets):
                #如果生成的对抗样本合法化后依然是对抗样本。
                d_d = x1x-X_soc_test[sample_n]
                L0 = np.linalg.norm(d_d,ord=0)
                print('\nL0=',L0)
                break#生成成功退出循环
    
    
        #JSMA更新样本
        gg=[]
        # gg = np.append(d[0][0:34],c)
        gg = d[0][0:78]
        idx, pix_sign=saliency_map(gg, mask)
        # print(idx, pix_sign, x1.shape)
        x1 = np.reshape(x1,[78])
        x1[idx]+=pix_sign * 0.1 * (max_[idx] - min_[idx]) 
        
        #达到极限的点不再参与更新
        if (x1[idx]<=min_[idx]) or (x1[idx]>=max_[idx]):
            mask[0][idx]=0
            x1[idx]=np.clip(x1[idx], min_[idx], max_[idx])
        
        #对x1的特征进行一个限制，限制在所有样本最大值内。
        x1 = np.reshape(x1,[1,78])
        x1 = scaler.inverse_transform(x1)
        x1x = []
        for i in range(78):
            x1x1 = np.float32(np.clip(x1[0][i:i+1],x1_d[i],x1_u[i]))
            x1x  =  np.append(x1x,x1x1)
        x1  = np.reshape(x1x,[1,78])
        x1 = scaler.transform(np.reshape(x1x,[1,78]))

    if epoch == 99:
        print('对抗样本生成失败，迭代次数 =',epoch, '\n')
        return False
    else:
        print('对抗样本生成成功，迭代次数 =',epoch,' , L0 =',L0, '\n')
        return x1x

In [ ]:
def FGSM(sample_n, max_iter=5):
    # 选择样本并且转换
    targets = [0,1]
    sample = np.array(X_train[sample_n:sample_n+1])
    x1 = np.reshape(sample,[1,78])

    # 每列最大最小值
    x1_d = np.min(X_soc, axis=0)
    x1_u = np.max(X_soc, axis=0)
    
    for epoch in range(max_iter):
        #计算梯度
        x_ = np.reshape(x1,[-1,78,1])
        # 用x_替换input_生成新的数据
        y1,_,gg_ = model.session.run([softmax_,loss_,gradw],{input_:x_,target_:[targets]})
        
        if np.argmax(y1) == np.argmax(targets):
            #如果生成了对抗样本。
            x1,x1x = x1_transform(x1)
            #将生成的对抗样本合法化
            x_ = np.reshape(x1,[-1,78,1])
            y1,_,gg_ = model.session.run([softmax_,loss_,gradw],{input_:x_,target_:[targets]})
            
            if np.argmax(y1) == np.argmax(targets):
                #如果生成的对抗样本合法化后依然是对抗样本。
                d_d = x1x-X_soc_test[sample_n]
                # print(d_d)
                L0 = np.linalg.norm(d_d,ord=0)
                print('\nL0=',L0)
                break #生成成功退出循环

            
        # 计算更改样本
        #  x1 = x1 + 1.1*signed_grad
        gg = []
        gg = gg_[0][0:78]
        gg = np.reshape(gg,[1,78])
        perb = np.sign(gg)
        x1 = x1 + 2 * perb

        # 对x1的特征进行一个限制，限制在所有样本最大值内
        x1 = scaler.inverse_transform(x1)
        x1x = []
        for i in range(78):
            x1x1 = np.float32(np.clip(x1[0][i:i+1],x1_d[i],x1_u[i]))
            x1x  =  np.append(x1x,x1x1)  
        x1  = np.reshape(x1x,[1,78])
        x1 = scaler.transform(np.reshape(x1x,[1,78]))

    if epoch == 4:
        print('对抗样本生成失败，迭代次数 =',epoch, '\n')
        return False
    else:
        print('对抗样本生成成功，迭代次数 =',epoch,' , L0 =',L0, '\n')
        return x1x


In [ ]:
def Gen_baleful_examples(ad_func, start_num, end_num=None):
    
    positive = np.int32(np.where(y_train==1)).reshape(-1)#正样本位置
    negative = np.int32(np.where(y_train==0)).reshape(-1)#负样本位置
    #输入负样本的位置，生成对抗样本。不是所有样本都能生成对抗样本，大部分可以。
    # print(negative)
    
    if end_num == None:
        end_num = start_num
        
    ae_list = [] 
    print(ad_func)
    nums = 0
    t_num = 0
    for i in range(start_num, end_num):
        
        if nums >= 10000:
            break
            
        pos = negative[i]
        test_x = X_soc[pos].reshape(78,)
        ae = ad_func(pos)    
        
        if type(ae) is np.ndarray:
            ae_list.append(ae)
            nums+=1

        else:
            # print("对抗失败的负样本位置", i)
            pass
        t_num += 1
    if end_num != start_num:
        print('生成率：{:.2%}'.format(len(ae_list) / (t_num)))
        
    return ae_list
    
import time
start = time.time()
ae_list = Gen_baleful_examples(Deepfool, start_num=0, end_num=100000)
end = time.time()
print (end - start)

In [ ]:
import matplotlib.pyplot as plt

def normalization2gray(data):
    _range = np.max(data) - np.min(data)
    return 255 * (data - np.min(data)) / _range

def save_png(sample):

    # 归一化
    npy = normalization2gray(sample)
    # 四舍五入取整
    img = np.rint(npy)
    # print(img)
    # 变形
    img = img.reshape((9, 9))
    # print(img)
    
    # 展示图片
#     plt.imshow(img, cmap='gray')
#     plt.show()

    return img 
    

In [ ]:
# 批量保存产生的对抗样本
def save_examples(ae_list, save_dir, ad_func_name):
    # print(sample.shape)
    
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
        
    info_txt = open(os.path.join(save_dir, f"{ad_func_name}-baleful.txt"), 'w')
    # 以恶意样本为原本生成对抗样本
    for i in range(len(ae_list)):
        
        # 位置信息保存
        png_path = os.path.join(save_dir, f"{ad_func_name}-baleful-{i}.png")
        info_str = f'{png_path} 0\n' # 0代表 baleful恶意样本
        info_txt.write(info_str)    
        
        # npy格式对抗样本
        ae = ae_list[i]
        sample = np.append(ae, [0, 0, 0])
        
        # 转换成img格式
        img = save_png(sample)
        
        # 保存图片
        plt.imsave(png_path, img, cmap='gray')
        
    info_txt.close()     
    print('Save Finish！')
    
# save_examples(ae_list,'.\DeepFool_pngs', 'Deepfool')
np.save(".\DeepFool_1w.npy", ae_list)
print("save .npy done")

In [ ]:
negative = np.int32(np.where(y_train==0)).reshape(-1)#负样本位置
print('negative',negative,len(str(negative)),negative.shape)

In [ ]:
def view_sample(sample):
    """
    epoch代表第几次迭代的图像
    sample为我们的采样结果
    """
    fig, axes = plt.subplots(figsize=(9,9), nrows=1, ncols=1, sharey=True, sharex=True)
    ax = axes
    img = sample # 这里sample代表生成的图像结果
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    im = ax.imshow(img.reshape((9,9)), cmap='Greys_r')
    
    return fig, axes

In [ ]:
ae = ae_list[0]
sample = np.append(ae, [0,0,0])
view_sample(sample)

In [ ]:
# 复制过来的结果

#原始恶意样本
se = [57420.0 ,1718.0 ,6.0 ,43.0 ,1.0 ,1.0 ,0.0 ,6.0 ,0.0 ,0.0 ,0.0 ,0.0 ,6.0 ,6.0 ,6.0 ,0.0 ,43.0 ,0.0 ,43.0 ,43.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,40.0 ,20.0 ,23255.81395 ,23255.81395 ,0.0 ,6.0 ,2.0 ,3.464101615 ,12.0 ,0.0 ,0.0 ,0.0 ,1.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.0 ,3.0 ,0.0 ,6.0 ,40.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.0 ,0.0 ,1.0 ,6.0 ,29200.0 ,0.0 ,0.0 ,40.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0]

# 由DeepFool白盒攻击算法生成的恶意对抗样本
deepfool_ae = [65535.0 ,0.0 ,7.64 ,7907271.5 ,79.33 ,1674.26 ,3000.83 ,331326.97 ,376.95 ,0.0 ,64.42 ,111.8 ,232.84 ,11.66 ,129.99 ,0.0 ,-13.0 ,0.0 ,4333376.5 ,2078429.0 ,5820347.0 ,1387770.62 ,0.0 ,3090121.5 ,3410977.25 ,1358718.38 ,1473928.0 ,183327.69 ,1060969.5 ,1546550.5 ,0.02 ,0.0 ,0.0 ,0.0 ,1799528.0 ,390757.16 ,131820.67 ,0.0 ,0.0 ,111.86 ,1.15 ,0.0 ,0.0 ,0.0 ,0.02 ,0.0 ,0.86 ,0.0 ,0.06 ,0.0 ,0.0 ,0.69 ,0.0 ,66.03 ,122.1 ,1784553.75 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,84.91 ,3007.71 ,1677.15 ,326482.81 ,32786.21 ,6592.64 ,0.0 ,60.0 ,51610.84 ,5918.59 ,24319.35 ,12805.21 ,1018882.62 ,0.0 ,1241427.5 ,844242.06]

# 由FGSM白盒攻击算法生成的恶意对抗样本
fgsm_ae = [59529.0 ,8007.0 ,9.96 ,60429088.0 ,1434.51 ,1913.51 ,11051.29 ,8458395.0 ,1946.59 ,324.95 ,510.61 ,798.5 ,8383.31 ,6.0 ,1304.76 ,3820.12 ,-13.0 ,0.0 ,35722104.0 ,5796347.0 ,60203112.0 ,18682142.0 ,0.0 ,36126660.0 ,17840556.0 ,109838048.0 ,17915184.0 ,15971842.0 ,27321182.0 ,17383810.0 ,0.0 ,0.0 ,0.0 ,0.0 ,24.06 ,19.98 ,806732.81 ,184654.31 ,93.42 ,8987.46 ,1380.91 ,2961.9 ,5.53 ,0.35 ,0.0 ,0.0 ,1.0 ,0.0 ,0.53 ,0.0 ,0.0 ,1.65 ,1526.18 ,510.61 ,2615.52 ,24.06 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1434.51 ,11051.29 ,1913.51 ,8458037.0 ,65535.0 ,15994.84 ,0.0 ,24.02 ,0.0 ,1487122.88 ,0.0 ,0.0 ,31073712.0 ,0.0 ,35185872.0 ,29279670.0]

# 由JSMA白盒攻击算法生成的恶意对抗样本
jsmas_ae = [65535.0 ,65534.0 ,12.8 ,43.0 ,1.0 ,1.0 ,0.0 ,6.0 ,0.0 ,0.0 ,0.0 ,0.0 ,6.0 ,6.0 ,6.0 ,0.0 ,43.0 ,0.0 ,43.0 ,43.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,-0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,40.0 ,20.0 ,23255.81 ,23255.81 ,0.0 ,6.0 ,2.0 ,3.46 ,12.0 ,0.0 ,0.0 ,0.0 ,1.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.0 ,3.0 ,0.0 ,6.0 ,40.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.0 ,0.0 ,1.0 ,6.0 ,29200.0 ,0.0 ,0.0 ,40.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0]

#由GAN训练恶意对抗样本生成的恶意对抗样本
gan_ae = [65467.08, 0.0, 17.0, 37.07, 1.0, 0.0, 0.0, 0.0, 0.0, 26.93, 0.59, 0.0, 30.89, 62.34, 23.96, 0.0, -13.0, 0.0, -5.85, -13.0, 0.0, 0.0, 0.0, 3.58, 0.0, 96.56, 0.0, 17067.28, 21.46, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 83175.84, 136.55, 27.13, 1.37, 49.49, 0.21, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.94, 224.44, 0.57, 89.26, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.43, 0.0, 521.33, -1.0, -1.0, 0.0, 13.33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.58]

In [ ]:
# 从结果中可以看到由白盒攻击算法生成的恶意对抗样本 和 由GAN训练恶意对抗样本生成的恶意对抗样本都被白盒检测系统认定为正常，说明攻击成功

def predict_rate(ae):
    # 数据处理
    ae = np.reshape(se, (1,78))
    ae = scaler.transform(ae)
    ae = np.reshape(se, (78,1))

    # 模型预测
    ae_pred = model.predict([ae])
    print('恶意样本/正常样本可能性')
    print(ae_pred[0])

    
sample = ae_list[0] 
predict_rate(sample)

In [10]:
ae_list = np.load('.\wpgan-gp_1b.npy')

def Avg_predict_rate(ae_list):
    lens = len(ae_list)
    ae_list = np.reshape(ae_list, [lens, 78])
    ae_list = scaler.transform(ae_list)
    ae_list = np.reshape(ae_list, [lens, 78,1])
    ae_pred = model.predict(ae_list)

    avarage = 0
    for i in ae_pred:
        avarage += i[1]
    
    rate = avarage / lens
    print("平均欺骗率：{:.2%}".format(rate))

Avg_predict_rate(ae_list)

平均欺骗率：99.74%
